In [12]:
import cv2
import numpy as np
from facenet_pytorch import MTCNN, InceptionResnetV1
from PIL import Image
import torch

# Load the pre-trained models
mtcnn = MTCNN(keep_all=True, device='cuda' if torch.cuda.is_available() else 'cpu')
resnet = InceptionResnetV1(pretrained=None).eval()
import torch

def get_face_embeddings(img):
    img_np = np.array(img)
    faces = mtcnn(img_np)

    if faces is None:
        return None

    if isinstance(faces, list):
        faces = faces[0]
    embeddings_list = []

    for face in faces:
        # Convert the face tensor to grayscale and resize to (160, 160)
        face_np = (face.permute(1, 2, 0).numpy() * 255).astype(np.uint8)
        img_gray = Image.fromarray(face_np[:, :, 0], mode='L')
        img_gray = img_gray.resize((160, 160))

        # Convert grayscale image back to RGB
        img_rgb = Image.merge("RGB", (img_gray, img_gray, img_gray))
        img_rgb_np = np.array(img_rgb) / 255.0
        img_rgb_tensor = torch.tensor(img_rgb_np.transpose(2, 0, 1), dtype=torch.float32).unsqueeze(0)
        embeddings = resnet(img_rgb_tensor).detach().numpy()

        # Append the embedding to the list
        embeddings_list.append(embeddings)

    # Return the list of embeddings
    return embeddings_list
def calculate_distance(emb1, emb2):
    # Calculate the Euclidean distance between two embeddings
    return np.linalg.norm(emb1 - emb2)

def recognize_face(input_embeddings, database_embeddings, threshold=1.0):
    # Compare the input embeddings with the embeddings in the database
    min_distance = float('inf')
    recognized_name = None

    for name, db_embedding in database_embeddings.items():
        distance = calculate_distance(input_embeddings, db_embedding)
        if distance < min_distance and distance <= threshold:
            min_distance = distance
            recognized_name = name

    return recognized_name
if __name__ == "__main__":
    # Load the image
    image_path = r"C:\Users\Ivar\Documents\Python Scripts\Face-Images(1)\Face Images\Final Training Images\face2\image_0024_Face_1.jpg"
    img = Image.open(image_path)

person1_image_path = r"C:\Users\Ivar\Documents\Python Scripts\Face-Images(1)\Face Images\Final Training Images\face2\image_0023_Face_1.jpg"
person2_image_path =r"C:\Users\Ivar\Documents\Python Scripts\Face-Images(1)\Face Images\Final Training Images\face2\image_0023_Face_1.jpg"

img = Image.open(person1_image_path)
img2 = Image.open(person1_image_path)
person1_face_np = np.array(img) / 255.0
person2_face_np = np.array(img2) / 255.0

person1_face_tensor = torch.tensor(person1_face_np.transpose(2, 0, 1), dtype=torch.float32).unsqueeze(0)
person2_face_tensor = torch.tensor(person2_face_np.transpose(2, 0, 1), dtype=torch.float32).unsqueeze(0)

# Get the face embeddings using the face recognition model
person1_embeddings = resnet(person1_face_tensor).detach().numpy()
person2_embeddings = resnet(person2_face_tensor).detach().numpy()

# Create the database of known face embeddings

input_embeddings = get_face_embeddings(img)

if input_embeddings is not None:

        database_embeddings = {
           'person1': person1_embeddings,
           'person2': person2_embeddings,

}
        recognition_threshold = 0.7
        recognized_name = recognize_face(input_embeddings, database_embeddings, threshold=recognition_threshold)
        if recognized_name is not None:
            print(f"Recognized: {recognized_name}")
        else:
            print("Unknown person.")
else:
        print("No face detected in the image.")


Recognized: person1
